In [2]:
import pandas as pd
import numpy as np
import math

import irc6_2007 as irc6

from Cross import Cross_section,cables, expansion_calc
from bridge_specs import box
# from allinput import main_input

In [3]:
import matplotlib.pyplot as plt

In [80]:
df=pd.read_excel('data/span.xlsx',index_col=None,header=None)
fck = 45 #input grade of concrete
span = int(df.iloc[2])
no = int(df.iloc[1])
sc=[(j)/(no-1)*span for j in range(no)]

***required:***
1. list of sections with cable profile
2. BM due to permanent dead load
3. BM due to superimposed dead load (without surfacing)
4. BM due to surfacing
5. BM due to live load (max of all vehicle loads)

**defining sections**

main section

In [130]:

"""INPUTS SECTION"""

"""FOR CROSS SECTION"""
length=[]
height=[]


#####################################################################################################################

"""GUI INPUT OF BRIDGE SECTION LENGTHS AND HEIGHTS"""

if False:
    main_input()

df=pd.read_excel('data/box.xlsx',index_col=None,header=None)


height=df.values.tolist()[0]
length=df.values.tolist()[1]


section=Cross_section(length,height)
cable_no = 18 #input no of cables
cable = cables(fck,cable_no,section)

Generating intermediate sections

In [131]:
sections = []
for i in range(no):  
    section1=Cross_section(length,height)
    section1.expansion_width=expansion_calc(span,sc[i],cable)
    section1.cableprop=cable.cablepos(sc[i],50)
    sections.append(section1)

In [132]:
# endsec = sections[0]
# endsec.cable_prop

# midsec = sections[4]
# midsec.cable_prop, endsec.Centroid

**Importing load**

In [133]:
df = pd.read_excel('outputs/Moments.xlsx', index_col=0)

#bm multipied by distribution factor
bm_pdl = np.array(df['Dead Load'])
bm_odl = np.array(df['Other Loads'])
bm_surf = np.array(df['Surface Loads'])
bm_pedl = np.array(df['Pedestrian Load'])

dfl = pd.read_excel('outputs/loads.xlsx', index_col=[0, 1]) #live load

**live load calculation**

Impact factor

In [134]:
vehicles = [irc6.ll_A, irc6.ll_70R, irc6.ll_70RT]
IF = [irc6.impact(i.name, span) for i in vehicles]
arr_if = np.array(IF)
arr_with_if = np.array(IF)+1 #including impact factor

distribution factor

In [135]:
dfe = pd.read_excel('outputs/max_e.xlsx', index_col=0)

ecc = dfe['MaxEccentricity'] #eccentricity

def df(ecc, webcc):
    df = (webcc/2+ecc)/webcc
    return round(df, 3)

webcc = 3 #input web center/center distance
distf = [df(i, webcc) for i in ecc]
distf

[1.067, 0.5, 0.635, 0.617]

bm for live load combinations

In [136]:
arr_maxbm = dfl.loc['MaxBM'].to_numpy() # max bm due to each veh
arr_maxbmif = arr_maxbm*arr_with_if[:, np.newaxis] # max bm including impact due to ll

arr_ll_comb = dfe.drop(columns='MaxEccentricity').to_numpy() # ll combinations in array

all_ll = arr_ll_comb.dot(arr_maxbmif) # ll with impact for each combination
all_unfactored_ll = all_ll #without dist factor
# all_unfactored_ll = all_ll*np.array(distf)[:, np.newaxis] # distributed bm due to live load including impact (bm*if*df)

bm_ll = np.amax(all_unfactored_ll, axis=0)
bm_ll

array([    0.      ,  5250.7152  ,  8922.6336  , 11142.723712,
       12024.9024  , 11619.922688,  9417.0208  ,  5428.332288,
           0.      ])

### Cable profile

In [137]:
# def cable_profile(end_cable_pos, mid_cable_pos, span_length, division=8):
#     cable_profile = []
#     x = [(j)/division*span_length for j in range(division+1)]
#     for i in x:
#         cable_pos_x = []
#         for j in range(len(end_cable_pos)):
#             h = end_cable_pos[j][1]-mid_cable_pos[j][1]
#             a = end_cable_pos[j][1]
#             y = a + 4*h*i/span_length - 4*h*i**2/span_length**2
#             cable_pos_x.append((end_cable_pos[j][0], y))
#         cable_profile.append(cable_pos_x)
#     return cable_profile

### Eccentricity of cables

In [138]:
def eccentricity(cable_positions, centroid):
    cable_areaxpos_sum = 0
    for i in cable_positions:
        cable_areaxpos_sum+=(i[1]-centroid)
    return cable_areaxpos_sum/len(cable_positions)

### Stress

In [139]:
list_of_sections = sections
prestressing_force = 2730 #input
prestressing_force_total = prestressing_force*cable_no
# prestressing_force_withloss = 0.75*prestressing_force

In [140]:
def tau(section, bm):
    tau_top = bm*section.ymin/section.I[0]
    tau_bot = -bm*section.ymax/section.I[0]
    return (tau_top, tau_bot)

presetressing stress

In [141]:
#due to eccentricity
bm_pre = [prestressing_force_total*eccentricity(i.cableprop, i.Centroid[1]) for i in list_of_sections] #bm due to full prestressing force

list_tau_pre = [tau(list_of_sections[index], bm_pre[index]) for index in range(len(list_of_sections))] #stress due to full prestressing force

#axial
list_tau_ax = [prestressing_force_total/sum(i.section_area) for i in list_of_sections]

list_tau_preTotal = [(list_tau_pre[index][0] + list_tau_ax[index], list_tau_pre[index][1] + list_tau_ax[index]) for index in range(len(list_of_sections))]

In [142]:
list_tau_preTotal

[(3277.460536860909, 8675.468950377986),
 (2666.411895480194, 9782.54906396105),
 (2229.9485802082545, 10573.32057366324),
 (1968.0705910450924, 11047.783479484551),
 (1880.7779279907045, 11205.937781424991),
 (1968.0705910450924, 11047.783479484551),
 (2229.9485802082545, 10573.32057366324),
 (2666.411895480194, 9782.54906396105),
 (3277.460536860909, 8675.468950377986)]

stress due to ded load

In [143]:
list_tau_DL = [tau(list_of_sections[index], bm_pdl[index]) for index in range(len(list_of_sections))]

In [144]:
list_tau_DL

[(0.0, -0.0),
 (1724.830588940131, -3124.9977743190043),
 (2956.852438183081, -5357.139041689721),
 (3696.0655477288515, -6696.42380211215),
 (3942.4699175774413, -7142.852055586292),
 (3696.0655477288515, -6696.42380211215),
 (2956.852438183081, -5357.139041689721),
 (1724.830588940131, -3124.9977743190043),
 (0.0, -0.0)]

stress due to surfacing

In [145]:
list_tau_surf = [tau(list_of_sections[index], bm_surf[index]) for index in range(len(list_of_sections))]

In [146]:
list_tau_surf

[(0.0, -0.0),
 (75.57281477393754, -136.9206225132493),
 (129.5533967553215, -234.72106716557025),
 (161.94174594415185, -293.4013339569628),
 (172.73786234042865, -312.961422887427),
 (161.94174594415185, -293.4013339569628),
 (129.5533967553215, -234.72106716557025),
 (75.57281477393754, -136.9206225132493),
 (0.0, -0.0)]

stress due to other load

In [147]:
list_tau_ODL = [tau(list_of_sections[index], bm_odl[index]) for index in range(len(list_of_sections))]

In [148]:
list_tau_ODL

[(0.0, -0.0),
 (114.50426480899627, -207.45548865643835),
 (196.29302538685076, -355.6379805538943),
 (245.36628173356345, -444.54747569236787),
 (261.72403384913434, -474.18397407185904),
 (245.36628173356345, -444.54747569236787),
 (196.29302538685076, -355.6379805538943),
 (114.50426480899627, -207.45548865643835),
 (0.0, -0.0)]

stress due to live load

In [149]:
list_tau_LL = [tau(list_of_sections[index], bm_ll[index]) for index in range(len(list_of_sections))]

In [150]:
arr_stress_top = np.array([
    list(zip(*list_tau_DL))[0], 
    list(zip(*list_tau_ODL))[0], 
    list(zip(*list_tau_surf))[0], 
    list(zip(*list_tau_LL))[0], 
    list(zip(*list_tau_preTotal))[0]
])

arr_stress_bot = np.array([
    list(zip(*list_tau_DL))[1], 
    list(zip(*list_tau_ODL))[1], 
    list(zip(*list_tau_surf))[1], 
    list(zip(*list_tau_LL))[1], 
    list(zip(*list_tau_preTotal))[1]
])

**Case 1: Rare combination I**

RI = DL + PS@10%loss

In [151]:
psf_RI = np.array([1, 0, 0, 0, 0.9]) #partial safety factor [DL, ODL, surf, LL, Ps] irc:6-2007 table B3

tauTop_RI = psf_RI.dot(arr_stress_top) #resultant pressure top

tauBot_RI = psf_RI.dot(arr_stress_bot) #resultant pressure bottom

tauTop_RI, tauBot_RI

(array([2949.71448317, 4124.60129487, 4963.80616037, 5467.32907967,
        5635.17005277, 5467.32907967, 4963.80616037, 4124.60129487,
        2949.71448317]),
 array([7807.92205534, 5679.29638325, 4158.84947461, 3246.58132942,
        2942.4919477 , 3246.58132942, 4158.84947461, 5679.29638325,
        7807.92205534]))

Allowable stresses

In [152]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.48*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-3276, 16800)

check

In [153]:
if all(tauTop_RI>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_RI<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


**Case 2: Rare combination II**

RII = DL + ODL + 1.2*Surf + 0.75*LL + PS@25%loss

In [154]:
psf_RII = np.array([1, 1, 1.2, 0.75, 0.75]) #partial safety factor [DL, ODL, surf, LL, PS] irc:6-2007 table B3

tauTop_RII = psf_RII.dot(arr_stress_top) #resultant pressure top

tauBot_RII = psf_RII.dot(arr_stress_bot) #resultant pressure bottom

tauTop_RII, tauBot_RII

(array([2458.09540265, 4183.53669698, 5412.19731176, 6150.2121475 ,
        6403.08547151, 6173.26958276, 5436.08525248, 4192.11884973,
        2458.09540265]),
 array([6506.60171278, 3380.49748024, 1154.44599947, -182.66776341,
        -640.81627943, -224.44255237, 1111.16652491, 3364.94858609,
        6506.60171278]))

Allowable stresses

In [155]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.36*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-3276, 12600)

check

In [156]:
if all(tauTop_RII>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_RII<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


**Case 2: Rare combination II**

RII = DL + ODL + 1.2*Surf + LL + PS@25%loss

In [157]:
psf_RII = np.array([1, 1, 1.2, 1, 0.75]) #partial safety factor [DL, ODL, surf, LL, PS] irc:6-2007 table B3

tauTop_RII = psf_RII.dot(arr_stress_top) #resultant pressure top

tauBot_RII = psf_RII.dot(arr_stress_bot) #resultant pressure bottom

tauTop_RII, tauBot_RII

(array([2458.09540265, 4268.10521161, 5555.90609073, 6329.67790738,
        6596.7596846 , 6360.42115438, 5587.75667837, 4279.54808195,
        2458.09540265]),
 array([6506.60171278, 3227.27871099,  894.07862349, -507.81859491,
        -991.70957222, -563.51831352,  836.37265741, 3206.54685213,
        6506.60171278]))

Allowable stresses

In [158]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.46*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-3276, 16100)

check

In [159]:
if all(tauTop_RII>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_RII<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


**Case 3: Permanent combination**

RII = DL + ODL + 1.2*Surf + 0.75*LL + PS@25%loss

In [160]:
psf_P = np.array([1, 1, 1.2, 0.75, 0.75]) #partial safety factor [DL, ODL, surf, LL, PS] irc:6-2007 table B3

tauTop_P = psf_P.dot(arr_stress_top) #resultant pressure top

tauBot_P = psf_P.dot(arr_stress_bot) #resultant pressure bottom

tauTop_P, tauBot_P

(array([2458.09540265, 4183.53669698, 5412.19731176, 6150.2121475 ,
        6403.08547151, 6173.26958276, 5436.08525248, 4192.11884973,
        2458.09540265]),
 array([6506.60171278, 3380.49748024, 1154.44599947, -182.66776341,
        -640.81627943, -224.44255237, 1111.16652491, 3364.94858609,
        6506.60171278]))

Allowable stresses

In [161]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.36*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-3276, 12600)

check

In [162]:
if all(tauTop_P>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_P<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


**Case 4: Quasi-Permanent**

QP = DL + ODL + PS@25%loss

In [163]:
psf_QP = np.array([1, 1, 0, 0, 0.75]) #partial safety factor [DL, ODL, surf, LL, PS] irc:6-2007 table B3

tauTop_QP = psf_QP.dot(arr_stress_top) #resultant pressure top

tauBot_QP = psf_QP.dot(arr_stress_bot) #resultant pressure bottom

tauTop_QP, tauBot_QP

(array([2458.09540265, 3839.14377536, 4825.60689873, 5417.48477275,
        5614.77739742, 5417.48477275, 4825.60689873, 3839.14377536,
        2458.09540265]),
 array([6506.60171278, 4004.458535  , 2217.213408  , 1144.86633181,
         787.41730641, 1144.86633181, 2217.213408  , 4004.458535  ,
        6506.60171278]))

Allowable stresses

In [164]:
tauTop_perm = -int(0.259*math.pow(fck, 2/3)*1000) #permissible stress on top (tensile) kN/m2

tauBot_perm = int(0.36*35*1000) #permissible stress on top (compression) kN/m2
tauTop_perm, tauBot_perm

(-3276, 12600)

check

In [165]:
if all(tauTop_QP>tauTop_perm) == True:
    print('top ok')
else:
    print('top not ok')
    
if all(tauBot_QP<tauBot_perm) == True:
    print('bottom ok')
else:
    print('bottom not ok')

top ok
bottom ok


In [166]:
tauTop_QP, tauBot_QP

(array([2458.09540265, 3839.14377536, 4825.60689873, 5417.48477275,
        5614.77739742, 5417.48477275, 4825.60689873, 3839.14377536,
        2458.09540265]),
 array([6506.60171278, 4004.458535  , 2217.213408  , 1144.86633181,
         787.41730641, 1144.86633181, 2217.213408  , 4004.458535  ,
        6506.60171278]))

### Ultimate strength Check

material property cable

In [167]:
fpk = 1860 # breaking stress
es = 195000 # modulus of elasticity

Adopted Stress Strain Curve for Ultimate Strength Check

In [174]:
ecu2 = 0.0035
ec2 = 0.0020

fp0 = fpk*0.87
fpd = fp0/1.15

e = fpd/es 

In [175]:

def stress(ec2, fck):
    """
    stress strain relationship. refer to IRC: 112-2020 Cl. 6.4.2.8
    :param ec2: strain
    :param fck: fck of concrete
    :return: stress for giver strain ec2 for concrete of grade fck
    """
    if ec2 < 0:
        stress = 0
    elif 0 <= ec2 <= 0.002:
        stress = 0.67*fck/1.5 * (1 - np.power((1 - ec2 / 0.002), 2))
    else:
        stress = 0.67*fck/1.5
    return stress


#### calculation of effective depth and xulim

##### effective depth
eff depth = depth - $\frac{\displaystyle\sum_{i=1} ^{j} y*n}{\displaystyle\sum_{i=1} ^{j} n}$

where,<br>
n = no of cables in layer i <br>
y = depth from bottom <br>
j represents total number of layer <br><br><br>

##### xu lim
$x_{u,\ lim\ }=\ \frac{\epsilon_{cu2}}{\epsilon_{cu2}+\ \epsilon}$<br>
where,<br>
$\epsilon$ = design strain = fpd/Es

In [207]:
depth = sections[0].position[3][1] + sections[0].height[3] - sections[0].position[1][1] #input dept from slab top to bottom mm
n = cable_no #input number of cables
effd = [] #effective depth at different sections
xulim = [] #limiting depth
for i in sections:
    cpx, cpy = zip(*i.cableprop)
    arr_y = np.array(list(set(cpy)))
    arr_n = np.array([cpy.count(i) for i in set(cpy)])
    ed = depth - sum(arr_y*arr_n)/n

    xul = ecu2/(ecu2+e)*ed
    
    effd.append(ed)
    xulim.append(xul)

Neutral axis
NA=$\ \frac{depth}{\epsilon_{cu2}\ -\ \epsilon_{bot}}\times\epsilon_{bot}$

In [208]:
etop = 0.0035
ebot = -0.02

na = -depth/(ecu2 - ebot)*ebot
na

2.723404255319149

In [209]:
sections[0].position[3][1] + sections[0].height[3] - sections[0].position[1][1]

3.2

In [194]:
sections[1].name

['left Pillar',
 'right pillar',
 'bottom slab',
 'top slab',
 'left cantilever',
 'right cantilever',
 'left triangle',
 'right triangle',
 'left top fillet',
 'right top fillet',
 'left bottom fillet',
 'right bottom fillet']

In [206]:
sections[6].height[3]

0.3